In [1]:
import extraction
import pickle

from sklearn.cluster import AgglomerativeClustering

In [2]:
# hyperparameters
n = 50  # number of articles
train_new = True  # create new (True) or use existing (False) pickle files

In [3]:
if train_new:
    df = extraction.load_data(n) # 3 articles

    pipeline = extraction.create_pipeline()
    raw_data = []
    for doc in df:    
        raw_data.append(pipeline(doc))  # list of lists of documents

    events = extraction.event_extraction(docs = raw_data, type = "Chambers")[1]

    pmi_matrix, flat_events, sorted_set = extraction.get_pmi_matrix(events)  # flat_events and sorted_set serve as the keys to the pmi matrix rows and columns

    # intermediate save 
    with open("flat_events_3.txt", "wb") as fp:   #Pickling
        pickle.dump(flat_events, fp)

    with open("sorted_set_3.txt", "wb") as fp:   #Pickling
        pickle.dump(sorted_set, fp)

    with open("pmi_matrix_3.txt", "wb") as fp:   #Pickling
        pickle.dump(pmi_matrix, fp)
    
else:
    with open("flat_events_3.txt", "rb") as fp:   # Unpickling
        flat_events = pickle.load(fp)

    with open("sorted_set_3.txt", "rb") as fp:   # Unpickling
        sorted_set = pickle.load(fp)

    with open("pmi_matrix_3.txt", "rb") as fp:   # Unpickling
        pmi_matrix = pickle.load(fp)


Identified 10654 eventive nouns.
Done 1 out of 1328
Done 501 out of 1328
Done 1001 out of 1328


In [4]:
# clustering
clustering = AgglomerativeClustering(n_clusters = 6, affinity='precomputed', linkage='complete').fit(pmi_matrix)
# add cluster labels to the sorted set
sorted_set = list(zip(sorted_set, clustering.labels_))

for x in range(6):
    print(f'CLUSTER {x}')
    for ev in sorted_set:
        if ev[1] == x:
            print(ev[0])

CLUSTER 0
BILLED
Browse
Click
Comparing
Continued
Drying
Editing
Expanding
Explaining
Focus
Includes
LABELED
Learn
Looking
Make
Reading
Seize
Smuggling
Stock
Think
Weeding
Working
abolished
absence
abused
accepted
accepting
access
accommodate
accused
achieved
acknowledging
action
activate
adapting
add
added
addicted
adding
addition
address
admit
admitted
admitting
adopt
advanced
aided
aim
alerted
alighting
allowed
analyzed
announcing
answer
apologize
appealed
appear
appreciate
approach
approval
approved
arrested
art
ask
asking
asserted
assessment
assigned
assortment
attack
attending
attitude
automate
avoid
back
ban
based
battle
bear
became
behavior
believe
believes
bind
binding
blathering
blocked
booked
boosts
born
bound
bow
bowl
break
bringing
brought
built
buying
called
calling
calls
campaign
carried
carrying
case
catapult
caught
celebrating
cessation
chainsaw
challenge
changed
characterized
charged
checked
claim
class
close
cobbled
coincided
coincidence
coinciding
coining
collaborat